In [1]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf


In [2]:
# replace all the total time and solve time by inf if its corresponding error is too large
function replace_byinf!(dic)
    for (key, value) in dic
        if findfirst("tot", key) == nothing && findfirst("solve", key) == nothing
            continue
        end
        splits = split(key, "_")
        solver = join(splits[1:length(splits) - 1], "_")
        for i in 1:length(value)
            if isnan(dic[string(solver, "_err")][i]) || dic[string(solver, "_err")][i] > 1e-8
                dic[key][i] = Inf
            end
        end
    end
end

replace_byinf! (generic function with 1 method)

In [3]:
@load "spe_all.jl.split02merge02.jld2"
dicAll = copy(dic)

Dict{Any, Any} with 84 entries:
  "lamg_build"        => [4.36221, 23.2889, 46.2032, 92.5763, 184.174]
  "acM2C2_err"        => [5.65863e-9, 2.80717e-9, 3.63631e-9, 2.76652e-9, 3.417…
  "hypre_solve"       => [0.58, 3.11, 6.12, 18.94, 38.15]
  "acM2C2_solve"      => [0.8393, 5.27383, 12.5595, 53.8824, 126.37]
  "icc2_tot"          => [2.19096, 14.1166, 27.249, 335.866, 676.257]
  "acM2C2_tot"        => [3.92265, 23.1905, 43.8394, 112.546, 257.258]
  "ac_lst_build"      => [2.77645, 14.6445, 28.9514, 60.193, 148.172]
  "petsc_hypre_build" => [2.17686, 10.9238, 21.7574, 44.1507, 89.5073]
  "lamg_its"          => [3.0, 4.0, 3.0, 21.0, 21.0]
  "lamg_solve"        => [1.50967, 10.8469, 14.3096, 303.567, 615.076]
  "hypre_its"         => [6.0, 6.0, 6.0, 10.0, 10.0]
  "cmg_solve"         => [0.647711, 2.68126, 5.54134, 18.7138, 37.7887]
  "ac_tot"            => [2.09137, 14.0933, 26.4701, 82.8764, 180.749]
  "ac1_solve"         => [0.97268, 5.68778, 14.1516, 65.5923, 149.548]
  "cmg_build"   

In [4]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        

#constant d = 3
ff(s) = (@sprintf "%.3g" sd(3,s))
nf(s) = (@sprintf "%d" sd(3,s))

ffK(s) = (@sprintf "%.3g" sd(3,s)/1000)
nfK(s) =  round(Int64, sd(3,s)/1000)

ffe8(s) = (@sprintf "%.3g" sd(3,s)*1e8)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
nfM(s) =  round(Int64, sd(3,s)/1e6)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)

ffe6 (generic function with 1 method)

In [5]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printSolverCell(d, solver, i)
    if haskey(d, string(solver, "_tot"))
        itr = d[string(solver, "_its")][i]
        ne = d["ne"][i]
        solveVn = d[string(solver, "_solve")][i] / ne
        totalVn = d[string(solver, "_tot")][i] / ne
        err = d[string(solver, "_err")][i]
        return string("&", nf(itr), "&", labelTvn(solveVn, err), "&", labelTvn(totalVn, err), "&",ffe8(err))
    else
        return "&N/A&N/A&N/A&N/A"
    end
end

function printOneLine(d, solvers, i)
    allCells = [printSolverCell(d, solver, i) for solver in solvers]
    return string(nfK(d["ne"][i]), "&", nfK(d["nv"][i]), join(allCells), "\\\\")
end

function printAllLines(d, solvers)
    for i in 1:length(d["testName"])
        println(printOneLine(d, solvers, i))
    end
end

printAllLines (generic function with 1 method)

In [6]:
solvers1 = ["ac", "ac-s2m2", "cmg2"]
printAllLines(dicAll, solvers1)

2910&422&17&0.24&0.718&0.433&13&0.23&1.07&0.288&15&0.306&1.07&0.709\\
14600&2100&22&0.359&0.967&0.337&14&0.291&1.31&0.315&15&0.294&0.882&0.644\\
28500&4100&22&0.404&0.928&0.474&14&0.337&1.34&0.416&15&0.312&0.873&0.957\\
55800&8000&39&0.893&1.49&0.495&27&0.796&1.79&0.256&23&0.52&1.1&0.981\\
112000&16000&41&1.03&1.62&0.331&26&0.868&1.91&0.28&23&0.531&1.1&0.663\\


In [7]:
solvers2 = ["hypre", "petsc_hypre", "icc2"]
printAllLines(dicAll, solvers2)

2910&422&6&0.199&0.508&0.0937&9&1.63&2.38&1e-06&38&0.675&0.752&0.882\\
14600&2100&6&0.213&0.518&0.378&9&1.71&2.46&4.33e-06&59&0.859&0.968&0.931\\
28500&4100&6&0.215&0.519&0.384&9&1.73&2.49&5.78e-06&57&0.843&0.955&0.938\\
55800&8000&10&0.34&0.646&0.287&9&1.77&2.57&0.00928&380&5.9&6.02&0.96\\
112000&16000&10&0.342&0.648&0.301&9&1.78&2.58&0.00939&383&5.92&6.06&0.992\\
